In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate


llm = ChatOpenAI(
    # model = "gpt-4o-mini",
    temperature = 0.2,
    streaming=True,
    )

memory = ConversationSummaryBufferMemory(
    llm=llm,
    memory_key="history",
    return_messages=True,
)

examples = (
    {
        "movie": "Top Gun",
        "answer": "🛩️👨‍✈️❤️‍🔥"
    },
    {
        "movie": "The Godfather",
        "answer": "🔫🧑‍🧑‍🧒‍🧒🍝"
    },
    {
        "movie": "Alien",
        "answer": "👽🔫🛸"
    },
)

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{answer}"),
        ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You must answer the movie with 3 emojis"),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm
 
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context({"input": question}, {"output": result.content})
    print(result)


In [2]:
invoke_chain("Sinister")

content='👻📹🔪'


In [3]:
invoke_chain("The witch")

content='🧙\u200d♀️🌲👹'


In [4]:
invoke_chain("interstellar")

content='🚀⏳🌌'


In [6]:
invoke_chain("What was the title of the movie you first asked?")

content='The first movie I asked about was "Sinister" 👻📹🔪'
